In [24]:
!pip install streamlit

!pip install pyngrok

In [25]:
%%writefile app.py

import streamlit as st
from PIL import Image
import torch
from torchvision import models, transforms
import torch.nn as nn
import os

# --- Config ---
st.set_page_config(page_title="SolarGuard - Solar Panel Classifier", layout="wide")

# --- Sidebar Navigation ---
page = st.sidebar.selectbox("📍 Navigation", ["Home", "Dashboard"])

# --- Load Model ---
@st.cache_resource
def load_model():
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 6)  # Assuming 6 classes
    model.load_state_dict(torch.load("/content/solarr_classification_model (4).pth", map_location=torch.device("cpu")))
    model.eval()
    return model

model = load_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# --- Image Preprocessing ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --- Class Labels ---
class_names = ['Bird Drop', 'Clean', 'Dusty', 'Electrical Damage', 'Physical Damage', 'Snow Covered']

# --- Home Page ---
if page == "Home":
    st.title("🔆 SolarGuard: Solar Panel Condition Classifier")
    st.markdown("""
    Welcome to **SolarGuard** — an AI-powered tool for detecting the condition of solar panels using deep learning.

    ### 🔍 Features
    - Upload images of solar panels
    - Classify their condition (e.g., clean, dusty, damaged)
    - Visual dashboard with predictions

    ### 🧠 Model
    - Backbone: ResNet-18
    - Fine-tuned on 6 condition classes
    - Integrated Grad-CAM support (coming soon!)

    ---
    👈 Use the sidebar to navigate to the **Dashboard** and try it out!
    """)

# --- Dashboard Page ---
elif page == "Dashboard":
    st.title("📸 Solar Panel Condition Dashboard")
    uploaded_file = st.file_uploader("Upload an image of a solar panel", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert('RGB')
        st.image(image, caption="Uploaded Image", use_container_width=True)

        # Preprocess
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Predict
        with torch.no_grad():
            output = model(input_tensor)
            _, pred = torch.max(output, 1)
            predicted_class = class_names[pred.item()]
            confidence = torch.nn.functional.softmax(output, dim=1)[0][pred.item()].item()

        # Display Result
        st.success(f"🧾 **Predicted Condition:** {predicted_class}")
        st.info(f"🔍 **Confidence:** {confidence:.2%}")


Overwriting app.py


In [26]:
# 1. Install Cloudflared
!pip install -q pyngrok
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 2. Run Streamlit in background
!streamlit run /content/app.py --server.port 8501 --server.address 0.0.0.0 &>/content/logs.txt &

# 3. Start Cloudflared tunnel
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


(Reading database ... 126384 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.8.0) over (2025.8.0) ...
Setting up cloudflared (2025.8.0) ...
Processing triggers for man-db (2.10.2-1) ...
2025-08-14T18:09:12Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-08-14T18:09:12Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-08-14T18:09:15Z INF +---------------------------------

In [ ]:
# 1. Install Cloudflared
!pip install -q pyngrok
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 2. Run Streamlit in background
!streamlit run /content/app.py --server.port 8501 --server.address 0.0.0.0 &>/content/logs.txt &

# 3. Start Cloudflared tunnel
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate